# loop on the OCs to estimate isochrone and mass

In [1]:
using PyCall

## PopStar modules...
synthetic= pyimport("popstar.synthetic")
evolution= pyimport("popstar.evolution")
atmospheres= pyimport("popstar.atmospheres")
reddening= pyimport("popstar.reddening")

using Distributions, Statistics, Distances
using DataFrames
using Random , Printf, Glob

import PyPlot , CSV
import Distances, StatsBase

rootdir = ENV["GAIA_ROOT"]

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/plotdens"
ocdir= "$wdir/oc2"
isodir= "$wdir/isochrones"

cd(wdir)

In [ ]:
cd(ocdir)
files= glob("*-oc.csv")
cd(wdir)

sample= rootdir*"/master/notebooks/data/BrowseTargets.18292.1530479692.fake"
df1= CSV.read(sample, delim= "|")
df1.name= strip.(df1.name)

noc= 0
for f in files
    fname= f[1:end-4] ; fvot= f[1:end-7] 
    ocfile= "$ocdir/$fname.csv" ; voname= "$fvot.vot"
    ocmassfile= "$ocdir/$fname-mass.csv"
    
    df= CSV.read(ocfile , delim= ";")
    
    t= split(fvot,"-")
    if length(t) == 3
        t[1]= t[1]*"-"*t[2]
    end
    ocname= t[1]
    
    println("## Name: $fname")
    logage= df1[(df1.name .== ocname), :log_age]
    # println(logage[1])
    
    ################################
    ## ISOCHRONES ##################
    ################################
    agmedian= median(df.ag[.! isnan.(df.ag)])
    distance= median(df.distance)
    nstar= length(df.distance)
    
    println("## N(star): $nstar")
    # Define isochrone parameters
    logAge= logage[1]  # Age in log(years)
    AKs= 0.0 # extinction in mags
    dist= distance # distance in parsec
    metallicity= 0.0 # Metallicity in [M/H]
    
    println("### log Age: $logAge")
    println("### AKs: $AKs")
    println("### distance: $dist")
    println("### metallicity: $metallicity")
    
    ## Define evolution/atmosphere models and extinction law
    if logAge>6.5
        evo_model = evolution.MergedBaraffePisaEkstromParsec() 
    else
        evo_model= evolution.MISTv1()
    end
    
    atm_func = atmospheres.get_merged_atmosphere
    red_law = reddening.RedLawHosek18b()

    ## Filter
    filt_list = [ "gaia,dr2_rev,Gbp", "gaia,dr2_rev,G"  ,"gaia,dr2_rev,Grp"]
    
    #####################################
    println("### Computing isochrones..")
    iso = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list, iso_dir= isodir)
    
    println("### Updating the DF and creating the new one with stellar mass..")
    
    data= iso.points
    println(data.keys())
    ndata= size(data)[1]

    mass= zeros(ndata)
    Gmag= zeros(ndata)
    BPmag= zeros(ndata)
    RPmag= zeros(ndata)
    color= zeros(ndata)

    for i in 1:ndata
        mass[i]= data[i][4]
        Gmag[i]= data[i][10]
        BPmag[i]= data[i][9]
        RPmag[i]= data[i][11]
        color[i]= BPmag[i]-RPmag[i]
    end

    ## Cluster
    BmR= df.bp .- df.rp
    GMAG= df.gbar + 5 .* log10.(df.distance) .- 17.
    

    ### Stellar mass 
    ################

    
    nxy= length(mass)
    A= Array{Float64}(undef,2,nxy)
    for i in 1:nxy
        A[1,i]= color[i]
        A[2,i]= Gmag[i]
    end
    
    d= Distances.Euclidean()
    mstar= zeros(length(BmR))

    for i in 1:length(BmR)
        starm= [BmR[i]; GMAG[i]]
        r= Distances.colwise(d, A, starm)
    
        idx= argmin(r)
        mstar[i]= mass[idx]
    end
    
    df.mass= mstar
    
    ## Some Logs mass stats...
    massmean= mean(mstar)
    println("### Mass (mean): $massmean")
    
    ### save the OC df
    CSV.write(ocmassfile, df, delim=";") 
    println("\n\n")
end
      

## Name: ASCC 10-2.7deg-oc
## N(star): 119
### log Age: 8.717
### AKs: 0.0
### distance: 661.0905129409848
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]


┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:78
└ @ Core ./In[2]:78
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:78
└ @ Core ./In[2]:78
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:79
└ @ Core ./In[2]:79
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:79
└ @ Core ./In[2]:79
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:80
└ @ Core ./In[2]:80
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:80
└ @ Core ./In[2]:80
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instea

### Mass (mean): 1.0398919579831933
